In [ ]:
import pandas as pd
import numpy as np
import requests
import json
from transliterate import translit
from tqdm import tqdm

In [ ]:
def get_tag(x, key_name):
    if str(key_name) in x:
        tmp = x[key_name]
        return tmp
    else:
        return 'NULL'

In [ ]:
city_bboxes = pd.read_csv('/data/russian_cities_bbox.csv')

In [ ]:
big_city = ['Нижний Новгород','Казань','Воронеж','Волгоград',
            'Уфа','Челябинск','Екатеринбург']

city_bboxes = city_bboxes[city_bboxes['name'].isin(big_city)]

In [ ]:
for ct in tqdm(city_bboxes['name'].index):
    city_name = translit(city_bboxes['name'].loc[ct], 'ru', reversed=True)
    print(city_name)
    tmp_bbox = city_bboxes['bbox_str'].loc[ct]
    
    overpass_url = "https://overpass.kumi.systems/api/interpreter"
    ###you can input other items, please see https://wiki.openstreetmap.org/wiki/Main_Page
    overpass_query = """
    [out:json];
    (node["amenity"]("""+tmp_bbox+""");
     node["shop"]("""+tmp_bbox+""");
     node["highway"="bus_stop"]("""+tmp_bbox+""");
    );
    out body;
    """
    response = requests.get(overpass_url, 
                            params={'data': overpass_query})
    data = response.json()

    loaded = pd.DataFrame(data['elements'])
    loaded['amenity'] = loaded['tags'].apply(lambda x: get_tag(x, 'amenity'))
    loaded['name'] = loaded['tags'].apply(lambda x: get_tag(x, 'name'))
    loaded['shop'] = loaded['tags'].apply(lambda x: get_tag(x, 'shop'))
    loaded['highway'] = loaded['tags'].apply(lambda x: get_tag(x, 'highway'))
    loaded.to_csv('name_{}_osm_poi.csv'.format(city_name), index=False)